In [2]:
import requests
import pandas as pd
import json
from dateutil import parser

In [3]:
API_KEY = "fb4c44655fe8a67467f67b6094ac3131-f3c5917cdf6b414289e70296954153fe"
ACCOUNT_ID = "101-001-27615226-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3/"

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [6]:
params = ({
    "count" : 10,
    "granularity" : "H1",
    "price" : "MBA" 
})

In [7]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url=url, params=None, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
json_data = response.json()

In [11]:
# data.keys()
instrument_list = json_data["instruments"]

In [12]:
len(instrument_list)
instrument_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [13]:
key_of_interest = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [14]:
instruments_dict = {}
for i in instrument_list:
    key = (i["name"])
    instruments_dict[key] = {k: i[k] for k in key_of_interest}
    

In [15]:
# instruments_dict

In [16]:
with open("../data/instruments.json", 'w') as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [29]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame.empty
    
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")
    


In [30]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [19]:
# create_data_file("USD_CAD", count-20, granularity="H1")


In [25]:

our_currency = ["EUR", "USD", "GBP", "JPY", "CHF", "NZD", "CAD", "AUD"]
for cp1 in our_currency:
    for cp2 in our_currency:
        cp = f"{cp1}_{cp2}"
        if cp in instruments_dict:
            for timeframe in["H1", "H4", "D"]:
                create_data_file(cp, count=4001, granularity=timeframe)

EUR_USD H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_USD H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_USD D 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_GBP H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_GBP H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_GBP D 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_JPY H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_JPY H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_JPY D 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CHF H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CHF H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CHF D 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_NZD H1 100 candles saved 202

In [27]:
our_curr = ["EUR", "USD", "GBP", "JPY", "CHF", "NZD", "CAD", "AUD"]
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_USD H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_GBP H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_GBP H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_JPY H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_JPY H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CHF H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CHF H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_NZD H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_NZD H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CAD H1 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_CAD H4 100 candles saved 2023-11-09 06:00:00+00:00 2023-12-01 18:00:00+00:00
EUR_AUD H1 100 candles saved